# Use PMML and Batch Deployments to predict iris species with `ibm-watsonx-ai`

This notebook contains steps from storing sample PMML model to starting scoring new data using batch deployment. 

Some familiarity with python is helpful. This notebook uses Python 3.10.

You will use a **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict iris species.

## Learning goals

The learning goals of this notebook are:

-  Working with the WML instance
-  Batch deployment of PMML model
-  Scoring of deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Model upload](#upload) 
3.	[Batch deployment creation](#deploy)
4.	[Scoring](#score)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [1]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [3]:
credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget
!pip install -U ibm-watsonx-ai | tail -n 1

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.

**Action**: Download sample PMML model from git project using wget.

In [7]:
import os
from wget import download

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = download('https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/models/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store downloaded file in Watson Machine Learning repository.

In [8]:
sw_spec_uid = client.software_specifications.get_uid_by_name("pmml-3.0_4.3")

meta_props = {
    client.repository.ModelMetaNames.NAME: "pmmlmodel",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1'}

In [9]:
published_model = client.repository.store_model(model=filename, meta_props=meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 3. Create battch deployment

You can use command bellow to create batch deployment for stored model.

In [11]:
model_id = client.repository.get_model_id(published_model)
deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Sample PMML Batch deployment",
        client.deployments.ConfigurationMetaNames.BATCH:{},
        client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
            "name": "S",
            "num_nodes": 1
        }
    }
)



#######################################################################################

Synchronous deployment creation for uid: '3284205c-d74a-4832-8c5e-3c398665c442' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af91a052-7a29-477d-b757-0a8e1fc73a61'
------------------------------------------------------------------------------------------------




Batch deployment has been created.

You can retrieve now your deployment ID.

In [12]:
deployment_id = client.deployments.get_id(deployment)

You can also list all deployments in your space.

In [ ]:
client.deployments.list()

If you want to get additional information on your deployment, you can do it as below.

In [ ]:
client.deployments.get_details(deployment_id)

<a id="score"></a>
## 4. Scoring

You can send new scoring records to batch deployment using by creating job.

In [14]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
            'values': [[5.1, 3.5, 1.4, 0.2]]
        }
    ]
}

job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)

Now, your job has been submitted to runtime.

You can retrieve now your job ID.

In [15]:
job_id = client.deployments.get_job_uid(job)

You can also list all jobs in your space.

In [ ]:
client.deployments.list_jobs()

If you want to get additional information on your job, you can do it as below.

In [ ]:
client.deployments.get_job_details(job_id)

**Monitor job execution**  
Here you can check status of your batch scoring.

In [18]:
import time

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    job_details_do = client.deployments.get_job_details(job_id)
    print(job_details_do)
else:
    print("Job hasn't completed successfully in 5 minutes.")

 Current state: completed
{'entity': {'deployment': {'id': 'af91a052-7a29-477d-b757-0a8e1fc73a61'}, 'platform_job': {'job_id': 'cfe636df-6651-496c-81e8-4486480f0708', 'run_id': 'd7e47aa5-58de-4d41-a65f-48d903ad1397'}, 'scoring': {'input_data': [{'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}], 'predictions': [{'fields': ['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species'], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}], 'status': {'completed_at': '2024-03-06T11:40:22.000Z', 'running_at': '2024-03-06T11:40:22.000Z', 'state': 'completed'}}}, 'metadata': {'created_at': '2024-03-06T11:40:08.000Z', 'id': 'f0960b8f-b5e6-4144-8af9-2eacbbb46599', 'modified_at': '2024-03-06T11:40:22.979Z', 'name': 'name_3801066c-96a5-4162-81e4-ff8af7a975ec', 'space_id': '93ee84d1-b7dd-42b4-b2ca-121bc0c86315'}}


**Get scored data**

In [19]:
import json

print(json.dumps(client.deployments.get_job_details(job_id), indent=2))

{
  "entity": {
    "deployment": {
      "id": "af91a052-7a29-477d-b757-0a8e1fc73a61"
    },
    "platform_job": {
      "job_id": "cfe636df-6651-496c-81e8-4486480f0708",
      "run_id": "d7e47aa5-58de-4d41-a65f-48d903ad1397"
    },
    "scoring": {
      "input_data": [
        {
          "fields": [
            "Sepal.Length",
            "Sepal.Width",
            "Petal.Length",
            "Petal.Width"
          ],
          "values": [
            [
              5.1,
              3.5,
              1.4,
              0.2
            ]
          ]
        }
      ],
      "predictions": [
        {
          "fields": [
            "$R-Species",
            "$RC-Species",
            "$RP-Species",
            "$RP-setosa",
            "$RP-versicolor",
            "$RP-virginica",
            "$RI-Species"
          ],
          "values": [
            [
              "setosa",
              1.0,
              1.0,
              1.0,
              0.0,
              0.0,
   

As we can see this is Iris Setosa flower.

<a id="cleanup"></a>
## 5. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Jan Sołtysik**, Software Engineer at IBM.

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning

Copyright © 2020-2024 IBM. This notebook and its source code are released under the terms of the MIT License.